In [1]:
%time

import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import pandas as pd
from env_vars import *

# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:


# def authorize():
#     token = util.prompt_for_user_token(username = SPOTIFYUSERNAME, 
#                                        scope = SCOPE, 
#                                        client_id = CLIENT_ID, 
#                                        client_secret = CLIENT_SECRET, 
#                                        redirect_uri = REDIRECT_URI)
#     if token:
#        sp = spotipy.Spotify(auth=token)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [3]:
# !pip install python-oath2

In [4]:
# import spotipy.oauth2 as oauth2

# sp_oauth = oauth2.SpotifyOAuth(client_id=CLIENT_ID,client_secret=CLIENT_SECRET,redirect_uri=REDIRECT_URI,scope=SCOPE)
# token_info = sp_oauth.get_cached_token() 
# if not token_info:
#     auth_url = sp_oauth.get_authorize_url()
#     print(auth_url)
#     response = input('Paste the above link into your browser, then paste the redirect url here: ')

#     code = sp_oauth.parse_response_code(response)
#     token_info = sp_oauth.get_access_token(code)

#     token = token_info['access_token']

# sp = spotipy.Spotify(auth=token)

In [5]:
# for k,v in sp.audio_features('54flyrjcdnQdco7300avMJ')[0].items():
#     print(k,v)

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
%time

def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
get_features('54flyrjcdnQdco7300avMJ')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


{'danceability': 0.693,
 'energy': 0.497,
 'key': 2,
 'loudness': -7.316,
 'mode': 1,
 'speechiness': 0.119,
 'acousticness': 0.679,
 'instrumentalness': 0,
 'liveness': 0.258,
 'valence': 0.473,
 'tempo': 81.308,
 'type': 'audio_features',
 'id': '54flyrjcdnQdco7300avMJ',
 'uri': 'spotify:track:54flyrjcdnQdco7300avMJ',
 'track_href': 'https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ',
 'duration_ms': 122067,
 'time_signature': 4}

In [4]:
%time

#Function to retrieve Spotify Track ID from Artist / Song via Spotipy

def get_spotify_id(artist,track):
    results = sp.search(q='artist:' + artist + ' ' + 'track:' + track, type='track')
    return results["tracks"]["items"][0]["id"]

get_spotify_id("Queen","We Will Rock You")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


'54flyrjcdnQdco7300avMJ'

In [5]:
song_list = pd.read_pickle("./song_list7.pkl")

In [6]:
song_list.shape

(178269, 4)

In [7]:
song_list2 = song_list[["artist","track"]]

In [8]:
import sqlite3

def create_table(name):
    conn = sqlite3.connect(f'./{name}.sqlite3')
    curs = conn.cursor()
    create_table = 'CREATE TABLE File (songid varchar(1000),artist varchar(1000),track varchar(1000), danceability float, energy float, key float, loudness float, mode float, speechiness float, acousticness float, instrumentalness float, liveness float, valence float, tempo float, duration_ms float,time_signature float);' 
    curs.execute(create_table)
    return conn

def insert(conn,f):    
    curs = conn.cursor()    
    query = (f'INSERT INTO File (songid, artist, track, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms,time_signature) VALUES (\"{f["songid"]}\",\"{f["artist"]}\",\"{f["track"]}\", {f["danceability"]}, {f["energy"]}, {f["key"]}, {f["loudness"]}, {f["mode"]}, {f["speechiness"]}, {f["acousticness"]},  {f["instrumentalness"]}, {f["liveness"]}, {f["valence"]}, {f["tempo"]}, {f["duration_ms"]},{f["time_signature"]});')
    curs.execute(query)
    curs.close()
    conn.commit()

In [9]:
def retrieve_track_features(rownumbers,tablename):
    feature_list = []
    conn = create_table(tablename)
    for entry,row in song_list2.head(rownumbers).iterrows():
        f = {}
        artist = row["artist"]
        track = row["track"]
        try:
            spotifyid = get_spotify_id(artist,track)
            features = get_features(spotifyid)
            f = {"songid":spotifyid,"artist":artist,"track":track,'danceability':features['danceability'],
                                 'energy':features['energy'],'key':features['key'],
                                 'loudness':features['loudness'],'mode':features['mode'],
                                'speechiness':features['speechiness'],'acousticness':features['acousticness'],
                                 'instrumentalness':features['instrumentalness'],'liveness':features['liveness'],
                                 'valence':features['valence'],'tempo':features['tempo'],'duration_ms':features['duration_ms'],'time_signature':features['time_signature']}
            try:
                insert(conn,f)
            except:
                print(artist,"Insert Statement failed")
        except:
            continue


retrieve_track_features(178269,"song_list7")

retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
Kenny "Blue" Ray Insert Statement failed
retrying ...1secs
retrying ...2secs
retrying ...3secs
Heino Insert Statement failed
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...2secs
retrying ...3secs
retrying ...2secs
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
Nicholas Hooper Insert Statement failed
retrying ...3secs
Myrick "Freeze" Guillory Insert Statement failed
retrying ...1secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
retrying ...3secs
retrying ...3secs
retrying ...